In [1]:
#pip install onnx onnxruntime
#pip install onnx
#!pip install onnx onnxruntime onnxruntime-tools

In [2]:
import joblib

X_scaler = joblib.load('data/scalers/X_scaler.save')
y_scaler = joblib.load('data/scalers/y_scaler.save')

In [3]:
import pandas as pd 
expert_agent = pd.read_csv("data/data_generation_mpc_110_190_6_all.csv")

In [4]:
X = expert_agent[["inflow", "height"]]
y = expert_agent[["speed1_rpm", "speed4_rpm"]]

In [5]:
X_scaled = X_scaler.transform(X)
y_scaled = y_scaler.transform(y)

In [6]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
import torch

seed = torch.Generator().manual_seed(42)

torch_dataset = TensorDataset(torch.tensor(X_scaled, dtype=torch.float32), 
                              torch.tensor(y_scaled, dtype=torch.float32))

dataset_loader = DataLoader(torch_dataset, batch_size=64, shuffle=True)

In [7]:
# Assuming your model takes input of the same shape as X_scaled
# Fetch a batch of data from the DataLoader
batch = next(iter(dataset_loader))

# The first part of the batch is your inputs (features)
dummy_input = batch[0]

In [8]:
import onnx
import onnxruntime

# Replace 'path_to_your_model.onnx' with the path to your ONNX model file
session = onnxruntime.InferenceSession('data/models/simple_model.onnx')

In [9]:
# Load your ONNX model
onnx_model = onnx.load('data/models/simple_model.onnx')

# Print input names
for input in onnx_model.graph.input:
    print(input.name)

onnx::Gemm_0


In [10]:
input_data = {'onnx::Gemm_0': dummy_input.numpy()}

In [11]:
yhat = session.run(None, input_data)[0]

pump1_rpm_hat, pump4_rpm_hat = yhat[:, 0], yhat[:, 1]

In [12]:
y_original = y_scaler.inverse_transform(batch[1].cpu())

pump1_rpm_opt, pump4_rpm_opt = y_original[:,0], y_original[:,1]